In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

input_folder = 'pictures'

preprocessed_images = []
original_with_contours = []
titles = []
debug_stages = []  # List of lists, one per image
cropped_cards = []  # List to store cropped card images

def display_image(image, title="Image"):
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title(title)
    plt.axis('off')
    plt.show()


for i in range(1, 20):
    
    filename = f'uno{i}.jpeg'
    path = os.path.join(input_folder, filename)

    image = cv2.imread(path)
    if image is None:
        print(f"Could not read {path}")
        continue
    
    max_width = 1280  
    if image.shape[1] > max_width:
        scale_ratio = max_width / image.shape[1]
        image = cv2.resize(image, (max_width, int(image.shape[0] * scale_ratio)))
    else:
        scale_ratio = 1.0 

    titles.append(filename)

    # Stage 1: Grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Stage 2: CLAHE
    clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(16, 16))
    equalized = clahe.apply(gray)

    # Add edge sharpening after CLAHE
    kernel_sharpen = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened = cv2.filter2D(equalized, -1, kernel_sharpen)

    # Stage 3: Blur (using GaussianBlur)
    blurred = cv2.GaussianBlur(equalized, (5, 5), 0)

    # Stage 4: Adaptive Thresholding (using sharpened image)
    adaptive_thresh = cv2.adaptiveThreshold(equalized, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                            cv2.THRESH_BINARY, 15, 5)

    # Stage 5: Morphological Closing (to close small holes in card edges)
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (4, 4))
    morphed = cv2.morphologyEx(adaptive_thresh, cv2.MORPH_CLOSE, kernel)

    # Stage 6: Canny Edge Detection
    canny_lower = 70
    canny_upper = 150
    canny = cv2.Canny(morphed, canny_lower, canny_upper)

    sobel_x = cv2.Sobel(morphed, cv2.CV_64F, 1, 0, ksize=3)
    sobel_y = cv2.Sobel(morphed, cv2.CV_64F, 0, 1, ksize=3)
    sobel_combined = cv2.magnitude(sobel_x, sobel_y)
    sobel_combined = cv2.convertScaleAbs(sobel_combined)

    sobel_combined = cv2.multiply(sobel_combined, 0.8)

    # Combine Canny and scaled Sobel edges
    combined_edges = cv2.bitwise_or(canny, sobel_combined)

    kernel_final = cv2.getStructuringElement(cv2.MORPH_RECT, (11, 11))
    refined_edges = cv2.morphologyEx(combined_edges, cv2.MORPH_CLOSE, kernel_final)

    # Save edge image for Montage 1
    edge_colored = cv2.cvtColor(refined_edges, cv2.COLOR_GRAY2RGB)
    preprocessed_images.append(edge_colored)

    # Contours for Montage 2 (fine-tuned filtering by area and aspect ratio)
    contours, _ = cv2.findContours(refined_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_img = image.copy()
    for cnt in contours:
        area = cv2.contourArea(cnt)
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / h

        # Fine-tuned thresholds
        if area < 1000 or aspect_ratio < 0.5 or aspect_ratio > 2.0:  # Adjusted thresholds
            continue

        # Simplify contour using approximation
        epsilon = 0.02 * cv2.arcLength(cnt, True)
        approx = cv2.approxPolyDP(cnt, epsilon, True)

        # Approximate full card position using minAreaRect
        rect = cv2.minAreaRect(cnt)
        box = cv2.boxPoints(rect)
        box = np.int32(box)
        cv2.drawContours(contour_img, [box], -1, (0, 255, 255), 2)

        # Filter bounding boxes based on size
        width, height = rect[1]
        if width < 50 or height < 50:  # Ignore very small boxes
            continue

        # Approximate card size threshold
        card_area = width * height
        if card_area < 0.5 * max_width * max_width:  # Adjust threshold as needed
            continue

        # Crop the card region
        card_box = np.int0(cv2.boxPoints(rect))
        x_min = max(0, min(card_box[:, 0]))
        x_max = min(image.shape[1], max(card_box[:, 0]))
        y_min = max(0, min(card_box[:, 1]))
        y_max = min(image.shape[0], max(card_box[:, 1]))
        cropped_card = image[y_min:y_max, x_min:x_max]

        # Save or display cropped cards
        if cropped_card.size > 0:
            cropped_cards.append(cropped_card)
            plt.figure()
            plt.imshow(cv2.cvtColor(cropped_card, cv2.COLOR_BGR2RGB))
            plt.title(f"Cropped Card from {filename}")
            plt.axis('off')
            plt.show()

        # Draw contours and convex hulls
        cv2.drawContours(contour_img, [approx], -1, (0, 255, 0), 2)
        hull = cv2.convexHull(cnt)
        cv2.drawContours(contour_img, [hull], -1, (255, 0, 0), 2)

    contour_img_rgb = cv2.cvtColor(contour_img, cv2.COLOR_BGR2RGB)
    original_with_contours.append(contour_img_rgb)

    # Save debug stages
    stages = [
        cv2.cvtColor(gray, cv2.COLOR_GRAY2RGB),
        cv2.cvtColor(equalized, cv2.COLOR_GRAY2RGB),
        cv2.cvtColor(blurred, cv2.COLOR_GRAY2RGB),
        cv2.cvtColor(adaptive_thresh, cv2.COLOR_GRAY2RGB),
        edge_colored
    ]
    debug_stages.append(stages)



# --- Display Montage 1: Preprocessed (edges) ---
cols = 3
rows = (len(preprocessed_images) + cols - 1) // cols
plt.figure(figsize=(4 * cols, 4 * rows))
for idx, img in enumerate(preprocessed_images):
    plt.subplot(rows, cols, idx + 1)
    plt.imshow(img)
    plt.title(titles[idx], fontsize=8)
    plt.axis('off')
plt.suptitle("Montage 1: Preprocessed Edge Images", fontsize=16)
plt.tight_layout()
plt.show()

# --- Display Montage 2: Contours on Original Images ---
plt.figure(figsize=(4 * cols, 4 * rows))
for idx, img in enumerate(original_with_contours):
    plt.subplot(rows, cols, idx + 1)
    plt.imshow(img)
    plt.title(titles[idx], fontsize=8)
    plt.axis('off')
plt.suptitle("Montage 2: Detected Contours on Original Images", fontsize=16)
plt.tight_layout()
plt.show()

# --- Display Cropped Cards ---
if cropped_cards:
    plt.figure(figsize=(10, 10))
    cols = 3
    rows = (len(cropped_cards) + cols - 1) // cols  # Calculate rows dynamically
    for idx, card in enumerate(cropped_cards):
        plt.subplot(rows, cols, idx + 1)
        plt.imshow(cv2.cvtColor(card, cv2.COLOR_BGR2RGB))
        plt.title(f"Card {idx + 1}")
        plt.axis('off')
    plt.suptitle("Cropped Cards", fontsize=16)
    plt.tight_layout()
    plt.show()
else:
    print("No cropped cards to display.")

stage_titles = ['Gray', 'Equalized', 'Blurred', 'Morphed', 'Edges']

plt.figure(figsize=(20, 4 * len(debug_stages)))
for i, stages in enumerate(debug_stages):
    for j, img in enumerate(stages):
        plt.subplot(len(debug_stages), 5, i * 5 + j + 1)
        plt.imshow(img)
        if i == 0:
            plt.title(stage_titles[j])
        plt.ylabel(titles[i], fontsize=8)
        plt.axis('off')
plt.suptitle("Preprocessing Stages Per Image", fontsize=16)
plt.tight_layout()
plt.show()
